In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import callbacks

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
from tslearn.metrics import dtw
from tslearn.barycenters import softdtw_barycenter
from tslearn.preprocessing import TimeSeriesScalerMinMax, TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, classification_report, precision_score, confusion_matrix
import numpy as np
import pandas as pd

/wsu/home/el/el85/el8550/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
np.random.seed(42)
cols = ['eeg c3', 'eeg c4', 'eeg cz', 'eeg f3', 'eeg f4', 'eeg f7', 'eeg f8', 'eeg fp1', 'eeg fp2', 'eeg fz', 'eeg o1', 'eeg o2', 'eeg p3', 'eeg p4', 'eeg pz', 'eeg t3', 'eeg t4', 'eeg t5', 'eeg t6']
# cols = ['eeg c3', 'eeg c4', 'eeg cz', 'eeg f3', 'eeg f4', 'eeg f7', 'eeg f8', 'eeg fp1', 'eeg fp2', 'eeg fz', 'eeg o1', 'eeg o2', 'eeg p3', 'eeg p4', 'eeg pz', 'eeg t3', 'eeg t4', 'eeg t5', 'eeg t6', 'resp effort', 'ecg ekg']

In [4]:
def splitDataFrameIntoSmaller(df, chunkSize = 256): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf

In [5]:
def extract_features(df):
    #load the data
    X, y = df[cols].values, df['label'].values[0]
    return(X, y)

In [6]:
def print_evaluation(y_sample, labels):
    print(str("Accuracy: {}").format(accuracy_score(y_sample, labels)))
    print(str("Precision: {}").format(precision_score(y_sample, labels)))
    print(str("Recall: {}").format(recall_score(y_sample, labels)))
    print(str("F1-Score: {}").format(f1_score(y_sample, labels)))
    print(confusion_matrix(y_sample, labels))
    print("\n")

In [7]:
def evaluate(X_train_res, y_train, X_test_res, y_test):

    #evaluate training
    print("Training Results:")
    # y_trai = np.asarray(y_train)

    y_pred = model.predict(X_train_res)
    pred_label=[]
    for elm in y_pred:
        if elm[0]>0.5:
            pred_label.append(1)
        else:
            pred_label.append(0)
    print_evaluation(pred_label, y_train)
    
    #evalulate testing
    print("Testing Results:")
    y_pred = model.predict(X_test_res)
    pred_label=[]
    for elm in y_pred:
        if elm[0]>0.5:
            pred_label.append(1)
        else:
            pred_label.append(0)

    print_evaluation(pred_label, y_test)

In [8]:
callback_arr = [callbacks.EarlyStopping(monitor='binary_accuracy',
                                        verbose=1,
                                        mode='min',
                                        patience=3)]
model = Sequential()
model.add(LSTM(100, return_sequences = True, input_shape = (256, len(cols))))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(25))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [12]:
def create_dataset(files):
    X = []
    y = []
    
    for file in files:
        df_raw = pd.read_csv(file)
        df_arr= splitDataFrameIntoSmaller(df_raw, chunkSize=256)
        for df in df_arr:
            try:
                Xr,yr = extract_features(df)
            #     X_scaled = TimeSeriesScalerMinMax().fit_transform(X.T)
                X.append(Xr)
                y.append(yr)
            except:
                pass
    return(X,y)

In [10]:
#get all the prepared csv files
import glob
filepath = "/wsu/home/groups/eeg/eeg-data/train/*.csv"
files = glob.glob(filepath)
print(files)

['/wsu/home/groups/eeg/eeg-data/train/0.csv', '/wsu/home/groups/eeg/eeg-data/train/1.csv', '/wsu/home/groups/eeg/eeg-data/train/2.csv', '/wsu/home/groups/eeg/eeg-data/train/3.csv', '/wsu/home/groups/eeg/eeg-data/train/4.csv', '/wsu/home/groups/eeg/eeg-data/train/5.csv', '/wsu/home/groups/eeg/eeg-data/train/6.csv', '/wsu/home/groups/eeg/eeg-data/train/7.csv', '/wsu/home/groups/eeg/eeg-data/train/8.csv', '/wsu/home/groups/eeg/eeg-data/train/9.csv', '/wsu/home/groups/eeg/eeg-data/train/10.csv', '/wsu/home/groups/eeg/eeg-data/train/11.csv', '/wsu/home/groups/eeg/eeg-data/train/12.csv', '/wsu/home/groups/eeg/eeg-data/train/13.csv', '/wsu/home/groups/eeg/eeg-data/train/14.csv', '/wsu/home/groups/eeg/eeg-data/train/15.csv', '/wsu/home/groups/eeg/eeg-data/train/16.csv', '/wsu/home/groups/eeg/eeg-data/train/17.csv', '/wsu/home/groups/eeg/eeg-data/train/18.csv', '/wsu/home/groups/eeg/eeg-data/train/19.csv', '/wsu/home/groups/eeg/eeg-data/train/20.csv', '/wsu/home/groups/eeg/eeg-data/train/21.csv

In [15]:
X,y = create_dataset(files)

In [16]:
#split into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,random_state=42)

X_train_res = np.asarray(X_train).reshape(len(X_train), 256, len(cols))
y_train = np.asarray(y_train)
X_test_res = np.asarray(X_test).reshape(len(X_test), 256, len(cols))
y_test = np.asarray(y_test)

In [ ]:
model.fit(X_train_res, y_train, validation_data=(X_test_res, y_test), epochs=10, callbacks=callback_arr)

Train on 322260 samples, validate on 80565 samples
Epoch 1/10
  5440/322260 [..............................] - ETA: 1:27:32 - loss: 0.3325 - binary_accuracy: 0.8963

In [ ]:
evaluate(X_train_res, y_train, X_test_res, y_test)

In [14]:
# # serialize model to JSON
# model_json = model.to_json()
# with open("/wsu/home/groups/eeg/eeg-data/model/model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("/wsu/home/groups/eeg/eeg-data/model/model.h5")
# print("Saved model to disk")